#  测试目标检测性能

In [ ]:
!pip install gluoncv

In [1]:
import gluoncv as gcv
import mxnet as mx

In [2]:
import os

class DetectionDataset(gcv.data.VOCDetection):
    CLASSES = ['cocacola', 'noodles', 'hand', 'fake']  # Yolo3 need at least 4 classes (https://github.com/apache/incubator-mxnet/pull/17689/files)
    def __init__(self, root):
        self._im_shapes = {}
        self._root = os.path.expanduser(root)
        self._transform = None
        self._items = [(self._root, x.strip('.jpg')) for x in os.listdir(self._root) if x.endswith('.jpg')]
        self._anno_path = os.path.join('{}', '{}.xml')
        self._image_path = os.path.join('{}', '{}.jpg')
        self.index_map = dict(zip(self.classes, range(self.num_class)))
        self._label_cache = self._preload_labels()
        
    def __str__(self):
        detail = self._root
        return self.__class__.__name__ + '(' + detail + ')'
    
    @property
    def classes(self):
        return self.CLASSES
    
    @property
    def num_class(self):
        return len(self.classes)
    
    def get_image_list(self):
        return [os.path.join(x[0], x[1] + '.jpg') for x in self._items]
        
test_dataset = DetectionDataset('../images/shenzhen_v1')
print('class_names:', test_dataset.classes)
print('num_images:', len(test_dataset))

class_names: ['cocacola', 'noodles', 'hand', 'fake']
num_images: 149


# 载入训练好的模型

In [3]:
net = gcv.model_zoo.get_model('yolo3_darknet53_custom', classes=test_dataset.classes, pretrained_base=False)
param_files = ([x for x in os.listdir('.') if x.endswith('.params')])
selected = param_files[0]
print('磁盘上有训练好的模型：', param_files)
net.load_parameters(selected)
print('载入完毕:', selected)

磁盘上有训练好的模型： ['object_detector_epoch200_11_05_2020_16_58_31.params']
载入完毕: object_detector_epoch200_11_05_2020_16_58_31.params


# 观察检测性能

In [4]:
images = test_dataset.get_image_list()
print('测试图像集：', images)

测试图像集： ['../images/shenzhen_v1/frame_1604547918.0878625.jpg', '../images/shenzhen_v1/frame_1604547990.0257223.jpg', '../images/shenzhen_v1/frame_1604547928.366902.jpg', '../images/shenzhen_v1/frame_1604547862.5851333.jpg', '../images/shenzhen_v1/frame_1604547908.8368647.jpg', '../images/shenzhen_v1/frame_1604557258.0419793.jpg', '../images/shenzhen_v1/frame_1604547889.3031857.jpg', '../images/shenzhen_v1/frame_1604547887.2466564.jpg', '../images/shenzhen_v1/frame_1604547868.7498348.jpg', '../images/shenzhen_v1/frame_1604557267.292567.jpg', '../images/shenzhen_v1/frame_1604547890.3314555.jpg', '../images/shenzhen_v1/frame_1604547944.8149817.jpg', '../images/shenzhen_v1/frame_1604547910.89245.jpg', '../images/shenzhen_v1/frame_1604547885.190378.jpg', '../images/shenzhen_v1/frame_1604547937.6198292.jpg', '../images/shenzhen_v1/frame_1604557269.347707.jpg', '../images/shenzhen_v1/frame_1604557268.3202035.jpg', '../images/shenzhen_v1/frame_1604547903.6937695.jpg', '../images/shenzhen_v1/fra

In [ ]:
%%time
from matplotlib import pyplot as plt
# Use GPU
ctx = mx.gpu(0)
# ctx = mx.cpu(0)
net.collect_params().reset_ctx(ctx)
for image in images:
    x, img = gcv.data.transforms.presets.yolo.load_test(image, short=512)
    class_IDs, scores, bounding_boxes = net(x.as_in_context(ctx))
    ax = gcv.utils.viz.plot_bbox(img, bounding_boxes[0], scores[0],
                         class_IDs[0], class_names=net.classes)
    plt.show()

# 测试结果总结

In [5]:
import time
from tqdm import tqdm
from gluoncv.data.batchify import Tuple, Stack, Pad
from gluoncv.data.transforms.presets.yolo import YOLO3DefaultTrainTransform

def validate(net, test_dataset, ctx):
    if isinstance(ctx, mx.Context):
        ctx = [ctx]
    size = len(test_dataset)
    metric = gcv.utils.metrics.voc_detection.VOC07MApMetric(iou_thresh=0.5, class_names=test_dataset.classes)
    net.collect_params().reset_ctx(ctx)
    metric.reset()
    width, height = 512, 512
    batch_size = 4
    batchify_fn = Tuple(Stack(), Pad(pad_val=-1))
    val_loader = mx.gluon.data.DataLoader(
        test_dataset.transform(YOLO3DefaultTrainTransform(width, height)), batchify_fn=batchify_fn,
        batch_size=batch_size, shuffle=False, last_batch='rollover', num_workers=0)
    with tqdm(total=size) as pbar:
        start = time.time()
        for ib, batch in enumerate(val_loader):
            data = mx.gluon.utils.split_and_load(batch[0], ctx_list=ctx, batch_axis=0, even_split=False)
            label = mx.gluon.utils.split_and_load(batch[1], ctx_list=ctx, batch_axis=0, even_split=False)
            det_bboxes = []
            det_ids = []
            det_scores = []
            gt_bboxes = []
            gt_ids = []
            gt_difficults = []
            for x, y in zip(data, label):
                ids, scores, bboxes = net(x)
                det_ids.append(ids)
                det_scores.append(scores)
                # clip to image size
                det_bboxes.append(bboxes.clip(0, batch[0].shape[2]))
                # split ground truths
                gt_ids.append(y.slice_axis(axis=-1, begin=4, end=5))
                gt_bboxes.append(y.slice_axis(axis=-1, begin=0, end=4))
                gt_difficults.append(y.slice_axis(axis=-1, begin=5, end=6) if y.shape[-1] > 5 else None)

            metric.update(det_bboxes, det_ids, det_scores, gt_bboxes, gt_ids, gt_difficults)
            pbar.update(batch[0].shape[0])
        end = time.time()
        speed = size / (end - start)
        print('Throughput is %f img/sec.'% speed)
    return metric.get()

final_result = validate(net, test_dataset, mx.gpu(0))

 99%|█████████▉| 148/149 [00:12<00:00, 11.94it/s]

Throughput is 12.021019 img/sec.


In [6]:
for name, score in zip(*final_result):
    print(name, ':', score)

cocacola : 0.9090909090909093
noodles : 0.9883040935672517
hand : 0.9042482238654488
fake : nan
mAP : 0.93388107550787
